# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [5]:
import re
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
#from StringIO import StringIO #for Python 2
from io import StringIO #for Python 3
import csv
import json
from pandas.io.json import json_normalize
#from flatten_json import flatten
from lxml import etree
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [6]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [7]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [8]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [9]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

    1.10 countries with the lowest infant mortality rates

In [58]:
# contruct DataFrame with infant_mortality for all countries
df = pd.DataFrame(np.random.randint(1,size=(len(root.findall('./country/infant_mortality')),2)),
                                    columns = ['country','infant_mortality'])
i = 0

for element in root.iterfind('country'):
    if(element.find('infant_mortality') != None):
        df.loc[i,'country'] = element.find('name').text
        df.loc[i,'infant_mortality'] = element.find('infant_mortality').text
        i = i+1
        #print('* ' + element.find('name').text + ': '+ element.find('infant_mortality').text)


In [59]:
df.head()

,country,infant_mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.9
3,Serbia,6.16
4,Andorra,3.69


In [60]:
#list of countries with lowest infant_mortality
df.sort_values('infant_mortality').head(10)

,country,infant_mortality
36,Monaco,1.81
28,Romania,10.16
142,Fiji,10.2
63,Brunei,10.48
124,Grenada,10.5
221,Mauritius,10.59
116,Panama,10.7
227,Seychelles,10.77
94,United Arab Emirates,10.92
105,Barbados,10.93


2.. 10 cities with the largest population


In [115]:
# contruct DataFrame with Year Vs Population for all cities

#creating dummy dataframe
data = np.random.randint(1,size=(len(root.findall('./country/city/population')),4))
columns = ['city','year','population','country']
df_ppl_ct = pd.DataFrame(data,columns=columns)

df_ppl_ct.head()
#print(len(df_ppl_ct), len(df_ppl))
#print(len(root.findall('./country/city/population')))

,city,year,population,country
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [117]:
i = 0
j = 0
k = 0
for element in root.iterfind("./country"):
    for subelement in element.iterfind("city"):
        for pplelement in subelement.iterfind("population"):
            df_ppl_ct.loc[k,'city'] = subelement.find('name').text
            df_ppl_ct.loc[k,'year'] = pplelement.get('year')
            df_ppl_ct.loc[k,'population'] = pplelement.text
            df_ppl_ct.loc[k,'country'] = element.find('name').text
            k=k+1
        j=j+1
    i=i+1

df_ppl_ct.head(15)

,city,year,population,country
0,Tirana,1987,192000,Albania
1,Tirana,1990,244153,Albania
2,Tirana,2011,418495,Albania
3,Shkodër,1987,62000,Albania
4,Shkodër,2011,77075,Albania
5,Durrës,1987,60000,Albania
6,Durrës,2011,113249,Albania
7,Vlorë,1987,56000,Albania
8,Vlorë,2011,79513,Albania
9,Elbasan,1987,53000,Albania


In [223]:
df_ppl_ct.year = pd.to_numeric(df_ppl_ct.year)
df_ppl_ct.population =pd.to_numeric(df_ppl_ct.population)
dfppl = pd.DataFrame(df_ppl_ct.groupby(['city']).year.max()).reset_index()

dfppl = dfppl.merge(df_ppl_ct,how = 'inner', on = ['city','year'])


print('2. 10 cities with the largest population: \n\n',dfppl.sort_values('population',ascending = False).head(10))

2. 10 cities with the largest population: 

                 city  year  population      country
315            Seoul  2010     9708483  South Korea
16        Al Qahirah  2006     8471859        Egypt
43           Bangkok  1999     7506700     Thailand
158        Hong Kong  2009     7055071    Hong Kong
157      Ho Chi Minh  2009     5968384      Vietnam
321        Singapore  2010     5076700    Singapore
9    Al Iskandariyah  2006     4123869        Egypt
253       New Taipei  2012     3939305       Taiwan
74             Busan  2010     3403135  South Korea
285        Pyongyang  2008     3255288  North Korea


3.. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [234]:
# contruct DataFrames for Year Vs Population  and ethnic_group data from all countries

#creating dummy dataframe for population
data = np.random.randint(1,size=(len(root.findall('./country/population')),3))
columns = ['country','year','population']
df_ppl = pd.DataFrame(data,columns=columns)

print(df_ppl.head())

#creating dummy dataframe for ethnic_group
data = np.random.randint(1,size=(len(root.findall('./country/ethnicgroup')),3))
columns = ['country','ethnic_grp','%population']
df_eth = pd.DataFrame(data,columns=columns)
print(df_eth.head())

   country  year  population
0        0     0           0
1        0     0           0
2        0     0           0
3        0     0           0
4        0     0           0
   country  ethnic_grp  %population
0        0           0            0
1        0           0            0
2        0           0            0
3        0           0            0
4        0           0            0


In [240]:
#Extrating population data from xml source.
i = 0
j = 0
for element in root.iterfind("./country"):
    for subelement in root[i].iterfind("population"):
        df_ppl.loc[j,'country'] = root[i].find('name').text
        df_ppl.loc[j,'year'] = subelement.get('year')
        df_ppl.loc[j,'population'] = subelement.text
        j = j+1
    i=i+1

print('\n population dataframe: \n\n', df_ppl.head(2))

#Extrating ethnic_group data from xml source.
i = 0
j = 0
for element in root.iterfind("./country"):
    for subelement in root[i].iterfind("ethnicgroup"):
        df_eth.loc[j,'country'] = root[i].find('name').text
        df_eth.loc[j,'ethnic_grp'] = subelement.text
        df_eth.loc[j,'%population'] = subelement.get('percentage')
        j = j+1
    i=i+1

print('\n\n\n ethnic_group dataframe: \n\n', df_eth.head(2))


 population dataframe: 

    country  year population
0  Albania  1950    1214489
1  Albania  1960    1618829



 ethnic_group dataframe: 

    country ethnic_grp %population
0  Albania   Albanian          95
1  Albania      Greek           3


In [233]:
#latest population data from countries
ppl = pd.DataFrame(df_ppl.groupby(['country']).year.max()).reset_index()
ppl = ppl.merge(df_ppl,on = ['country','year'],how='inner')
ppl.head()

,country,year,population
0,Afghanistan,2013,26023100
1,Albania,2011,2800138
2,Algeria,2010,37062820
3,American Samoa,2010,55519
4,Andorra,2011,78115


In [277]:
#Merging population and ethnic_group data frames

eth_wrld = df_eth.merge(ppl, on=['country'], how='inner')
eth_wrld.year = pd.to_numeric(eth_wrld.year)
eth_wrld['%population'] = pd.to_numeric(eth_wrld['%population'])
eth_wrld.population = pd.to_numeric(eth_wrld.population)
eth_wrld = eth_wrld.assign(eth_population = eth_wrld.population*(eth_wrld['%population']/100))
#pd.DataFrame(eth_wrld,eth_wrld.population*(eth_wrld['%population']/100)],
#                                       columns =list(eth_wrld.columns)+['eth_population'])
eth_wrld.head()

,country,ethnic_grp,%population,year,population,eth_population
0,Albania,Albanian,95.0,2011,2800138,2.660131e+06
1,Albania,Greek,3.0,2011,2800138,8.400414e+04
2,Greece,Greek,93.0,2011,10816286,1.005915e+07
3,Macedonia,Macedonian,64.2,2011,2059794,1.322388e+06
4,Macedonia,Albanian,25.2,2011,2059794,5.190681e+05


In [289]:
eth_wrld = eth_wrld.sort_values('ethnic_grp').reset_index(drop = True)

eth_wrld.head()

,country,ethnic_grp,%population,year,population,eth_population
0,Uganda,Acholi,4.0,2014,34856813,1.394273e+06
1,Ethiopia,Afar,1.7,2012,84320987,1.433457e+06
2,Djibouti,Afar,35.0,2010,834036,2.919126e+05
3,Andorra,African,5.0,2011,78115,3.905750e+03
4,Nigeria,African,99.0,2011,164294516,1.626516e+08


In [297]:
#Top 10 Ethnic groups in the world.
print('Top 10 Ethnic groups in the world: \n\n',
pd.DataFrame(eth_wrld.groupby(['ethnic_grp'])['eth_population'].sum()).reset_index().sort_values('eth_population',ascending = False).head(10).reset_index(drop = True)
     )

Top 10 Ethnic groups in the world: 

     ethnic_grp  eth_population
0  Han Chinese    1.245059e+09
1   Indo-Aryan    8.718156e+08
2     European    4.948722e+08
3      African    3.183251e+08
4    Dravidian    3.027137e+08
5      Mestizo    1.577344e+08
6      Bengali    1.467769e+08
7      Russian    1.318570e+08
8     Japanese    1.265342e+08
9        Malay    1.219936e+08


In [ ]:
# 

data = np.random.randint(1,size=(len(root.findall('./country/population')),3))
columns = ['country','year','population']
df_ppl = pd.DataFrame(data,columns=columns)

df_ppl.head()

4..  name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [329]:
#Extracting details of Longest river

longest_river = ""
long_rvr_country = ""
rvr_length = 0.0
for element in root.iterfind("river"):
    if element.find("length") != None:
        if float(element.find("length").text) > rvr_length:
            longest_river = element.find("name").text
            rvr_length = float(element.find("length").text)
            long_rvr_country = element.get("country")

print("\n4. a) logest river is: ",longest_river,", its length is ",rvr_length," and it is located in country(s): ",long_rvr_country)


#Extracting details of Largest lake
largest_lake = ""
large_lake_country = ""
lake_area = 0.0
for element in root.iterfind("lake"):
    if element.find("area") != None:
        if float(element.find("area").text) > lake_area:
            largest_lake = element.find("name").text
            lake_area = float(element.find("area").text)
            large_lake_country = element.get("country")

print("\n4. b) largest lake is: ",largest_lake,", its area is ",lake_area," and it is located in country(s): ",large_lake_country)


#Extracting details of airport at highest elevation
higth_airport = ""
aiport_country = ""
airport_elevation = 0.0
for element in root.iterfind("airport"):
    if element.find("elevation") != None:
        if element.find("elevation").text != None:
            if float(element.find("elevation").text) > airport_elevation:
                higth_airport = element.find("name").text
                airport_elevation = float(element.find("elevation").text)
                aiport_country = element.get("country")

print("\n4. c) airport at highest elevation is: ",higth_airport,", its elevation is ",airport_elevation," and it is located in country(s): ",aiport_country)
        



4. a) logest river is:  Amazonas , its length is  6448.0  and it is located in country(s):  CO BR PE

4. b) largest lake is:  Caspian Sea , its area is  386400.0  and it is located in country(s):  R AZ KAZ IR TM

4. c) airport at highest elevation is:  El Alto Intl , its elevation is  4063.0  and it is located in country(s):  BOL
